# Reshape Pandas Warmup

![](viz/pandas_exercise.gif)

In [1]:
# Run this cell w/o changes to load tests

from test_background import pkl_dump, test_obj_dict, run_test_dict, run_test

In [2]:
#__SOLUTION__

# Run this cell w/o changes to load tests

from test_background import pkl_dump, test_obj_dict, run_test_dict, run_test

## Imports

Import:
- Pandas as the alias pd

- Numpy as the alias np

- Matplotlib.pyplot as the alias plt

Run %matplotlib inline

In [3]:
#Your code here

In [4]:
#__SOLUTION__

#imports

#data manipulation
import pandas as pd
import numpy as np

#jn command to render matplotlib graphs inline
%matplotlib inline

#jn command to turn off klaxon alarms
import warnings
warnings.filterwarnings('ignore')

Read in the csv 'econ_stats' from the data folder as a dataframe

Assign it to the variable econ_stats

In [5]:
#Your code here

In [6]:
#__SOLUTION__

econ_stats = pd.read_csv('data/econ_stats.csv')

## Data Exploration

Take a look at `econ_stats.head()`, `econ_stats.info()` and `econ_stats.describe(include='all')`

How many unique values are in the categorical variables "Country" and "Stat"?  If some repeat, what are they?

How many unique values in the numerical variables "Year" and "Data"?  If some repeat, what are they?

How does the data appear to be organized?

In [7]:
#Your code here

In [8]:
#__SOLUTION__

print(econ_stats.head())
print(econ_stats.info())
print(econ_stats.describe(include='all'))

for column in econ_stats.columns:
    print(f'There are {len(econ_stats[column].unique())} unique values in {column}')

print(f'''
There are {len(econ_stats.Country.unique())} unique values in 'Country' {econ_stats.Country.unique()}

{len(econ_stats.Stat.unique())} in 'Stat' {econ_stats.Stat.unique()}

{len(econ_stats.Year.unique())} in 'Year' {econ_stats.Year.unique()}

and {len(econ_stats.Data.unique())} in 'Data'

For every year b/t 1956 and 1965, there are 5 countries and 3 statistics for which there is a 
unique piece of data
''')

  Country  Year                  Stat       Data
0      US  1956  Domestic Wheat Price  22.981728
1      US  1956         Wheat Exports  80.327631
2      US  1956         Wheat Imports  54.982899
3      US  1957  Domestic Wheat Price  50.507799
4      US  1957         Wheat Exports  54.024780
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
Country    150 non-null object
Year       150 non-null int64
Stat       150 non-null object
Data       150 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.8+ KB
None
       Country         Year                  Stat        Data
count      150   150.000000                   150  150.000000
unique       5          NaN                     3         NaN
top         US          NaN  Domestic Wheat Price         NaN
freq        30          NaN                    50         NaN
mean       NaN  1960.500000                   NaN   82.056109
std        NaN     2.881904            

## Data Reshaping

In order to make this data easier to manipulate and analyze, we want to turn it from "long" to "wide"

In other words, we want to make each row an observation which has:
- 1 country
- 1 year
- data values for each of the 3 stats



making a total of 5 columns

We already have "Country" and "Year", so let's work on making the additional 3 columns.

In class, we've gone over `.pivot()` to reshape.

We're going to make each of these three columns a different way.  

## Way 1: list comprehension 

To make the `Domestic Wheat Price` column, do the following:

- create a column named `Domestic Wheat Price` that is filled with nulls
- write a function that:
    - takes input parameters `row`, `df`
    - finds the `Year` value in `df` for that `row`
    - finds the `Country` in `df` for that `row`
    - finds the value of `Domestic Wheat Price` in `df` for those `Year` and `Country` values
    - returns that value
- make a list comprehension that uses your function to fill `Domestic Wheat Price`
- check to make sure `Domestic Wheat Price` no longer has nulls!

(notice that there are repeat values in `Domestic Wheat Price`, because we have multiple rows with the same country and year values)

In [9]:
#Your code here

In [10]:
#__SOLUTION__

econ_stats['Domestic Wheat Price'] = np.nan

econ_stats.head()

def fill_price(row, df):
    
    '''
    row-wise grabbing of 'Domestic Wheat Price' value
    
    Paramters: 
        row: specific row to grab information from
        df: dataframe w/ "Country", "Year", "Stat" and "Data" columns
            'Stat' must include "Domestic Wheat Price", with value in the "Data" column
            
    output: value of 'Stat' column for 'Domestic Wheat Price' for 
    values matching the Country and Year of given row
    
    
    '''
    row_year = df['Year'][row]
    
    row_country = df['Country'][row]
    
    mask = (
        (df['Year']==row_year) &
        (df['Country']==row_country) &
        (df['Stat']=='Domestic Wheat Price')
    )
    
    price = df[mask]['Data'].values[0]
    
    return price

econ_stats['Domestic Wheat Price'] = [
    fill_price(row, econ_stats) 
    for row 
    in econ_stats.index
]

print('There are nulls in "Domestic Wheat Price": {}'
      .format(
          econ_stats['Domestic Wheat Price']
          .isnull()
          .values
          .any()
      )
)

econ_stats.head()

# #used for tests
# pkl_dump([(
#     econ_stats['Domestic Wheat Price'], 'domestic_wheat_price_column'
# )])

There are nulls in "Domestic Wheat Price": False


,Country,Year,Stat,Data,Domestic Wheat Price
0,US,1956,Domestic Wheat Price,22.981728,22.981728
1,US,1956,Wheat Exports,80.327631,22.981728
2,US,1956,Wheat Imports,54.982899,22.981728
3,US,1957,Domestic Wheat Price,50.507799,50.507799
4,US,1957,Wheat Exports,54.024780,50.507799


In [27]:
#run this cell to check your work

run_test(econ_stats['Domestic Wheat Price'], 'domestic_wheat_price_column')

## Way 2: Split-Apply-Combine

To make the `Wheat Exports` column, do the following:

- create an empty list called `dfs`
- create a `for loop` cycling through the unique `Country` names
- inside the loop, for each country:
    - Assign to a variable `temp` all rows from `econ_stats` where:
        - the value in the `Country` column is the country in the current loop
        - the value in the `Stat` column is 'Wheat Exports'
    - Rename the `data` column to 'Wheat Exports'
    - drop extraneous columns (everything but `Country`, `Year` and `Wheat Exports`) from `temp`
    - Append `temp` to `dfs`
- check to make sure you have all the dataframes you expect to have in `dfs`
- assign to `wheat_exports` the [concatenation](https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.concat.html) of all the dataframes inside of `dfs` 
- merge `wheat_exports_frames` with `econ_stats`, using `Country` and `Year` as the keys
- sort `econ_stats` by `Country` and `Year`
- reset the index (this will become clearer why in the method below) without appending the old index as a column (ie make the `drop` parameter `True`)

In [12]:
#Your code here

In [13]:
#__SOLUTION__

dfs = []

for country in econ_stats.Country.unique():

    mask = (
        (econ_stats['Country']== country) &
        (econ_stats['Stat'] == 'Wheat Exports')
    )

    temp = econ_stats[mask]

    temp.rename(columns={'Data':'Wheat Exports'}, inplace=True)

    temp.drop(['Stat', 'Domestic Wheat Price'], axis=1, inplace=True)

    dfs.append(temp)
    
print(f'Expect 5 frames, have {len(dfs)}')
    
wheat_exports = pd.concat(dfs)

econ_stats = pd.merge(
    econ_stats, wheat_exports, 
    how='inner', 
    on=['Country', 'Year']
)

econ_stats.sort_values(['Country', 'Year'], inplace=True)

econ_stats.reset_index(inplace=True, drop=True)

# econ_stats = econ_stats[['Country', 'Year', 'Stat', 'Data', 'Domestic Wheat Price', 'Wheat Exports']]

econ_stats.head()

# #used for tests
# pkl_dump([(
#     econ_stats['Wheat Exports'], 'wheat_exports_column'
# )])

,Country,Year,Stat,Data,Domestic Wheat Price,Wheat Exports
0,China,1956,Domestic Wheat Price,81.933098,81.933098,56.343518
1,China,1956,Wheat Exports,56.343518,81.933098,56.343518
2,China,1956,Wheat Imports,49.741995,81.933098,56.343518
3,China,1957,Domestic Wheat Price,74.558164,74.558164,79.839552
4,China,1957,Wheat Exports,79.839552,74.558164,79.839552


In [26]:
#run this cell to check your work

run_test(econ_stats['Wheat Exports'], 'wheat_exports_column')

## Way 3: .where()

[`.where()`](https://pandas.pydata.org/pandas-docs/version/0.25.2/reference/api/pandas.Series.where.html) is a *fast* way to replace values of one series or dataframe based on boolean conditionals.

In our case, we want to make a new column `Wheat Imports` that:
- has the value of `Data` when `Stat` == `"Wheat Imports"`
- replaces the values of `Data` where `Stat` != `"Wheat Imports"` with the above value for the right `Country`/`Year` combination

`.where()` as a method takes two parameters:
- a conditional which returns `True` or `False`
- a replacement value when the conditional is `False`

So: if the condition is `True`, the original value is kept.  If `False`, the new value is substituted.  

Both the condition and the replacement can be a series or a dataframe.  If they are series/dataframes, when the condition is `False`, the value of the original series/dataframe is substituted with the value from the replacement series/dataframe at the same index.

*Ex: `a` = pd.Series([1,2,3,4,5,6]), `b` = pd.Series([0,-1,-2,-3,-4,-5])*

*`a.where(a%2==0, b)` = 0,2,-2,4,-4,6*

To make the `Wheat Imports` column:
- append the `.where()` method to econ_stats['Data']
- make the conditional in the first parameter that the `Stat` column is equal to `"Wheat Imports"`
    - if `True`, we'll get the `Data` value when `Stat` == `"Wheat Imports"`
- to create the series (the values of `Data` where `Stat`==`"Wheat Imports"`) to substitute when the conditional is `False`:
    - filter `econ_stats` to only show rows where `Stat` has the value of `Wheat Imports`
    - select the 'Data' column from that frame
    - [repeat](https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.Series.repeat.html) the values 3 times (why?)
    - reset the index (why?)
    - select the `Data` column from that frame
- remember to assign the `econ_stats['Data'].where()` expression you made to `econ_stats['Wheat Imports']`

Make sure the frame is still sorted by `Country` and `Year` when you're done

In [15]:
#Your code here

In [16]:
#__SOLUTION__

econ_stats['Wheat Imports'] = econ_stats['Data'].where(
    econ_stats['Stat'] == 'Wheat Imports',
    econ_stats
    [econ_stats['Stat']=='Wheat Imports']
    ['Data']
    .repeat(3)
    .reset_index()
    ['Data']
)

# #used for tests
# pkl_dump([(
#     econ_stats['Wheat Imports'], 'wheat_imports_column'
# )])
econ_stats.head()

,Country,Year,Stat,Data,Domestic Wheat Price,Wheat Exports,Wheat Imports
0,China,1956,Domestic Wheat Price,81.933098,81.933098,56.343518,49.741995
1,China,1956,Wheat Exports,56.343518,81.933098,56.343518,49.741995
2,China,1956,Wheat Imports,49.741995,81.933098,56.343518,49.741995
3,China,1957,Domestic Wheat Price,74.558164,74.558164,79.839552,62.476126
4,China,1957,Wheat Exports,79.839552,74.558164,79.839552,62.476126


In [25]:
#run this cell to test your work

run_test(econ_stats['Wheat Imports'], 'wheat_imports_column')

## Drop 

The `Stat` and `Data` columns are now redundant; drop 'em

We now have a bunch of duplicated rows

- Find the number of duplicated rows
- Drop the duplicated rows
- Make sure the resulting dataframe has the number of rows you expect

In [18]:
#Your code here

In [19]:
#__SOLUTION__
econ_stats.drop(['Stat', 'Data'], axis=1, inplace=True)

print(f'len before dropping {len(econ_stats)}')
print(f'number of dupes {econ_stats.duplicated().sum()}')

econ_stats.drop_duplicates(inplace=True)

print(f'len after dropping {len(econ_stats)}')

len before dropping 150
number of dupes 100
len after dropping 50


## Strrrretch goal: your turn

You may notice that we can continue the "widening" process further, by making columns for each `Country`'s data and having `Year` the only column left from our original frame

Let's do that

Create 15 new columns for each row, 3 of the data columns each for each of the five countries

Call each new column `"Country+existing data column name"`

Use whichever method you think is fastest, but apply the method dynamically (eg write the same code 15 times with some values changed)

Drop the `Country` column and the three data columns after you're done, so that the frame is just `Year` and the fifteen new data columns

In [20]:
#Your code here

In [21]:
#__SOLUTION__

dfs = []

for country in econ_stats.Country.unique():

    mask = (
        (econ_stats['Country']== country) 
    )

    temp = econ_stats[mask]

    temp.rename(
        columns=
        {column:country+' '+column 
         for column 
         in temp.columns[2:]}, 
        inplace=True
    )

    temp.drop('Country', axis=1, inplace=True)
    dfs.append(temp)
    
frame = dfs[0]
for new_frame in dfs[1:]:
    frame = pd.merge(frame, new_frame, how='inner', on='Year')

econ_stats = pd.merge(
    econ_stats, frame, 
    how='outer', 
    on=['Year']
)

econ_stats.sort_values(['Country', 'Year'], inplace=True)

econ_stats.reset_index(inplace=True, drop=True)

econ_stats.drop(
    ['Country', 'Domestic Wheat Price', 
     'Wheat Exports', 'Wheat Imports'
    ], 
    axis=1, 
    inplace=True
)

print(f'len before dropping: {len(econ_stats)}')
print(f'number of dupes: {econ_stats.duplicated().sum()}')
econ_stats.drop_duplicates(inplace=True)
print(f'len after dropping: {len(econ_stats)}')

econ_stats.head()

# #used for tests
# pkl_dump([(
#     econ_stats, 'stretch_goal'
# )])

len before dropping: 50
number of dupes: 40
len after dropping: 10


,Year,China Domestic Wheat Price,China Wheat Exports,China Wheat Imports,France Domestic Wheat Price,France Wheat Exports,France Wheat Imports,Soviet Union Domestic Wheat Price,Soviet Union Wheat Exports,Soviet Union Wheat Imports,UK Domestic Wheat Price,UK Wheat Exports,UK Wheat Imports,US Domestic Wheat Price,US Wheat Exports,US Wheat Imports
0,1956,81.933098,56.343518,49.741995,112.472808,137.834532,129.978449,34.082140,66.369536,49.033085,37.206487,87.307491,27.734788,22.981728,80.327631,54.982899
1,1957,74.558164,79.839552,62.476126,88.643140,112.045101,65.461591,125.591115,91.768343,101.981566,65.963443,26.634591,60.205639,50.507799,54.024780,51.734145
2,1958,134.687733,115.448149,99.996746,55.646619,108.453598,136.327167,108.848113,66.582251,74.202894,74.318569,26.855445,96.128557,100.431163,55.146884,8.744424
3,1959,103.992156,46.135510,98.383959,154.399830,64.940812,82.299856,119.030836,79.480691,66.575412,49.330824,78.337677,71.321469,81.066514,99.459582,39.571007
4,1960,83.759193,133.809948,136.716246,142.823510,83.228588,94.345567,67.750034,82.201482,90.913316,67.046024,37.165599,64.487817,8.157261,20.588848,63.653674


In [24]:
#run this cell to test your work
run_test(econ_stats, 'stretch_goal')